<a href="https://colab.research.google.com/github/Dhruv-Sharma01/ES335_Andrew_NG/blob/main/T4_Partc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 983.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import pandas as pd
from langchain_groq.chat_models import ChatGroq

# Use the Few-Shot prompting method using UCI-HAR dataset to predict the activities that you performed. Ensure that both your examples and test query undergo similar preprocessing. How did the model perform?

In [ ]:
df = pd.read_csv('output_X.csv')
df1 = pd.read_csv('output_Y.csv')
df = pd.concat([df, df1], axis=1)

print(df)

         accx    accy    accz           y_train
0      0.9883 -0.0447  0.0562  WALKING_UPSTAIRS
1      0.9854 -0.0481  0.0501  WALKING_UPSTAIRS
2      0.9956 -0.0640  0.0007  WALKING_UPSTAIRS
3      1.0039 -0.0706 -0.0000  WALKING_UPSTAIRS
4      1.0130 -0.0623  0.0090  WALKING_UPSTAIRS
...       ...     ...     ...               ...
28282  1.2234 -0.2779  0.0058           WALKING
28283  1.1741 -0.1879  0.0178           WALKING
28284  0.9345  0.0812  0.0816           WALKING
28285  0.8160  0.0433  0.1530           WALKING
28286  0.8697  0.0125  0.1740           WALKING

[28287 rows x 4 columns]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the datasets
df_X = pd.read_csv('output_X.csv')
df_Y = pd.read_csv('output_Y.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.2, random_state=42)

# Optionally, verify the shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (22629, 3), y_train shape: (22629, 1)
X_test shape: (5658, 3), y_test shape: (5658, 1)


In [ ]:
import random

# Define a mapping from activity labels to human-readable activity names
activity_labels = {
    "WALKING": 1,
    "WALKING_UPSTAIRS": 2,
    "WALKING_DOWNSTAIRS": 3,
    "SITTING": 4,
    "STANDING": 5,
    "LAYING": 6
}

# Sample a few random examples from the training set for few-shot learning
few_shot_examples = []
for _ in range(3):  # Choose how many examples you'd like to include
    random_index = random.randint(0, len(X_train) - 1)
    example_data = X_train.iloc[random_index].values
    example_label = y_train.iloc[random_index].values[0]
    few_shot_examples.append((example_data, activity_labels[example_label]))

# Construct the few-shot input prompt
few_shot_input_prompt = "You are an AI trained to classify human activities based on accelerometer data.\n"
few_shot_input_prompt += "Here are some examples of how accelerometer data is classified into specific human activities:\n\n"

for i, (data, label) in enumerate(few_shot_examples):
    few_shot_input_prompt += f"Example {i+1}:\n"
    few_shot_input_prompt += f"Data: {list(data)}\n"
    few_shot_input_prompt += f"Label: {label}\n\n"

few_shot_input_prompt += "Now, classify the following accelerometer data into the correct human activity.\n\n"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming the necessary imports and definitions are already in place
llm_classification_predictions = []
actual_activity_labels = []

# Iterate over each row in X_test
for index, row in X_test.iterrows():
    # Prepare the data string from the row
    test_data_string = list(row.values)

    # Construct the classification query
    classification_query = (f"{few_shot_input_prompt}\n\n"
                            f"Classify the following accelerometer data into the correct human activity.\n\n"
                            f"Data: {test_data_string}\n\n"
                            "Please provide the classification in the following format:\n"
                            "Label: <predicted_label>")

    # Initialize and use the language model
    language_model_name = "llama3-70b"  # Model name
    llm_model = ChatGroq(model=groq_models[language_model_name], api_key=Groq_token, temperature=0)
    llm_output = llm_model.invoke(classification_query)

    # Extract the LLM prediction
    llm_predicted_label = extract_label(llm_output.content.strip())

    # Store LLM prediction and actual label
    llm_classification_predictions.append(llm_predicted_label)

    # Use index alignment to get the correct label
    actual_activity_label = y_test.iloc[index] if index < len(y_test) else None
    actual_activity_labels.append(actual_activity_label.values[0] if actual_activity_label is not None else None)

# Convert lists to numpy arrays for plotting
predicted_values = np.array(llm_classification_predictions)
actual_values = np.array(actual_activity_labels)

# Step 3: Plot a Bar Graph

# Get unique activity labels and their counts
unique_labels, actual_counts = np.unique(actual_values, return_counts=True)
_, predicted_counts = np.unique(predicted_values, return_counts=True)

# Plotting the bar graph
bar_width = 0.35
index = np.arange(len(unique_labels))

plt.figure(figsize=(10, 6))
plt.bar(index, actual_counts, bar_width, label='Actual', color='b')
plt.bar(index + bar_width, predicted_counts, bar_width, label='Predicted', color='r')

plt.xlabel('Activity Labels')
plt.ylabel('Counts')
plt.title('Comparison of Actual and Predicted Activity Labels')
plt.xticks(index + bar_width / 2, [activity_labels[label] for label in unique_labels])
plt.legend()

plt.show()
